this script is used to test io workflow with neo4j

In [1]:
from neo4j import GraphDatabase
url="bolt://localhost:7687"
username="neo4j"
password="123./\.abc"

In [2]:
driver = GraphDatabase.driver(url, auth=(username, password))

In [3]:
create_example = """
CREATE (a:person {name: $name, description: $description})
"""
with driver.session() as session:
    session.run(create_example, name="tom", description="a person who like playing basketball.")
    session.run(create_example, name='jack', description="a person who like running.")

In [4]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/opt/homebrew/anaconda3/envs/kg/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertF

In [5]:
import torch
import torch.nn as nn

def tokenize_and_encode(sentences, tokenizer, model, max_batch_size=64, padding_length=128):
    """
    Tokenize and encode the sentences
    """
    encoded = []
    for i in range(0, len(sentences), max_batch_size):
        encoded_batch = tokenizer.batch_encode_plus(
            sentences[i : i + max_batch_size],
            padding="max_length",
            max_length=padding_length,
            truncation=True,
            return_tensors="pt",
        )
        # model outputs the last hidden states
        with torch.no_grad():
            model_output = model(**encoded_batch)
        batch_last_hidden_states = model_output.last_hidden_state
        encoded.append(batch_last_hidden_states.mean(dim=1))
    encoded = torch.cat(encoded, dim=0)
    return encoded